In [9]:
import re
import numpy as np
import pandas as pd
from utils.utils import get_project_root
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline


In [10]:
path_to_data = get_project_root() / "data"
df = pd.read_csv(path_to_data / "df.csv")
# turn date columns into datetime format
df[df.loc[:, df.columns.str.contains("date")].columns] = df.loc[:, df.columns.str.contains("date")].apply(
    pd.to_datetime)
# turn language and source_domain into categoricals
df[["language", "source_domain"]] = df[["language", "source_domain"]].astype("category")
# convert text-columns into text (from object
df[["description", "filename", "title", "title_page", "title_rss", "text_main"]].astype("str")

,description,filename,title,title_page,title_rss,text_main
0,Die Essayistin und Schriftstellerin Joan Didio...,aktuell_feuilleton_buecher_autoren_schriftstel...,Schriftstellerin Joan Didion mit 87 Jahren ges...,Schriftstellerin Joan Didion mit 87 Jahren ges...,Schriftstellerin Joan Didion mit 87 Jahren ges...,Die Essayistin und Schriftstellerin Joan Didio...
1,Heute schon Pigs Pigs Pigs Pigs Pigs Pigs Pigs...,aktuell_feuilleton_buecher_rezensionen_sachbuc...,"J.R. Moores' Buch ""Electric Wizards""","J.R. Moores' Buch ""Electric Wizards""","J.R. Moores' Buch ""Electric Wizards""",Heute schon Pigs Pigs Pigs Pigs Pigs Pigs Pigs...
2,"Auf einer neuen Dating-Seite suchen Einsame, d...",aktuell_feuilleton_dating-seite-fuer-impfgegne...,Dating-Seite für Impfgegner: Die ungeimpfte Liebe,Dating-Seite für Impfgegner: Die ungeimpfte Liebe,Unterwegs auf Impffrei:Love: Die ungeimpfte Liebe,"Die Menschen wollen lieben. Selbst die, die ha..."
3,Drehungen eines Unbelehrbaren: Nach Kritik an ...,aktuell_feuilleton_debatten_frank-ulrich-montg...,Frank Ulrich Montgomery verteidigt seine Richt...,Frank Ulrich Montgomery verteidigt seine Richt...,Montgomerys Mätzchen: Ihr seid ja nur eifersüc...,Ungern möchte man stören bei dem langen Lauf z...
4,Der Ethikrat befürwortet mehrheitlich eine all...,aktuell_feuilleton_debatten_impfpflicht-jetzt-...,Impfpflicht jetzt? Wir müssen miteinander reden!,Impfpflicht jetzt? Wir müssen miteinander reden!,Impfpflicht oder nicht: Wir müssen miteinander...,Als am 1. Januar 2019 Heribert Prantl in seine...
...,...,...,...,...,...,...
1252,"Das ""James Webb""-Teleskop ist erfolgreich ins ...",wissenschaft_video235869636_Weltraum-Teleskop-...,"Weltraum-Teleskop ""James Webb"": Astronaut Walt...","Weltraum-Teleskop ""James Webb"": Astronaut Walt...","""Da kann noch viel schiefgehen""","Das ""James Webb""-Teleskop ist erfolgreich ins ..."
1253,Eine Studie des Universitätsklinikums Hamburg-...,wissenschaft_video236043420_Studie-aus-Hamburg...,Studie aus Hamburg: Organschäden auch bei mild...,Studie aus Hamburg: Organschäden auch bei mild...,Organschäden auch bei mildem Corona-Verlauf,Eine Studie des Universitätsklinikums Hamburg-...
1254,Von Französisch-Guayana ins All: Das Weltraumt...,wissenschaft_weltall_james-webb-startet-seine-...,"""James Webb"" startet seine Mission",»James Webb« startet seine Mission - DER SPIEGEL,»James Webb« startet seine Mission,nan
1255,Nach gut 30 Jahren Planung war es so weit: Ein...,wissenschaft_weltall_james-webb-weltraumtelesk...,James-Webb-Weltraumteleskop ist ins All gestartet,James-Webb-Weltraumteleskop ist ins All gestar...,James-Webb-Weltraumteleskop ist ins All gestartet,»Hubble«-Nachfolger Weltraumteleskop »James We...


In [1]:
import math
from flair.data import Sentence
from flair.models import SequenceTagger

# load tagger
# WARNING - 2.24 GB model size!
tagger = SequenceTagger.load("flair/ner-german-large")
article_columns = ["title", "title_page", "title_rss", "description", "text_main"]
articles = {"texts": df[article_columns]}
entities = []
for article in [x[1] for x in articles["texts"].iterrows()]:
    entity = {}
    for k in article_columns:
        try:
            entity[k] = str(article[k]).replace("\x84", '"').replace("\x93", '"')
            chunks = [article[k]]
            if len(article[k]) > 512:
                # brute-force chunk the article
                chunks = [article[k][512 * (x - 1):512 * x] for x in range(1, math.ceil(len(article[k]) / 512))]
            chunks_ner = []
            for chunk in chunks:
                sentence = Sentence(chunk)
                tagger.predict(sentence)
                chunks_ner.append(sentence.to_dict(tag_type='ner'))
            entity[k + "_entities"] = chunks_ner
        except:
            pass
    entities.append(entity)
ent = pd.DataFrame(entities)
ent.to_csv("entities.csv", index=False)
ent

2022-01-09 18:42:33,223 loading file C:\Users\LeonWolf\.flair\models\ner-german-large\6b8de9edd73722050be2547acf64c037b2df833c6e8f0e88934de08385e26c1e.4b0797effcc6ebb1889d5d29784b97f0a099c1569b319d87d7c387e44e2bba48


NameError: name 'df' is not defined

In [11]:
df_ent = pd.read_csv(path_to_data / "entities.csv")

In [12]:
df_j = df.join(df_ent[[x for x in df_ent.columns if "entities" in x]])
df.head(1)

,title,title_page,title_rss,description,text_main,language,source_domain,filename,url,image_url,...,author_3,author_4,author_5,author_6,author_7,author_8,author_9,author_10,author_11,author_12
0,Schriftstellerin Joan Didion mit 87 Jahren ges...,Schriftstellerin Joan Didion mit 87 Jahren ges...,Schriftstellerin Joan Didion mit 87 Jahren ges...,Die Essayistin und Schriftstellerin Joan Didio...,Die Essayistin und Schriftstellerin Joan Didio...,de,www.faz.net,aktuell_feuilleton_buecher_autoren_schriftstel...,https://www.faz.net/aktuell/feuilleton/buecher...,https://media1.faz.net/ppmedia/aktuell/8915247...,...,Joachim Müller-Jung,Maja Brankovic,Dyrk Scherff,Friedrich Schmidt,Gerald Felber,Kim Maurus,NaN,NaN,NaN,NaN


In [13]:
entity_columns = [x for x in df_ent.columns if "entities" in x]
org_dict = {}
for entity in entity_columns:
    ent_list = []
    for x in [x for x in df_j[entity]]:
        try:
            tmp = re.sub("'labels': \[(\w{3,4}) \((\d+\.?\d*)\)]", "'\g<1>': \g<2>", x)
            tmp = eval(tmp)
            tmp_list = [y for x in tmp for y in x["entities"]]
        except:
            tmp_list = []
        ent_list.append(tmp_list)
    org_list = []
    for x in ent_list:
        tmp_list = []
        for y in x:
            if "ORG" in y:
                tmp_list.append(y["text"].strip())
        org_list.append(tmp_list)
    org_dict[entity] = org_list

In [18]:
df_j = df_j.join(pd.DataFrame(pd.Series(org_dict["title_entities"]).to_list()).add_prefix("org_").fillna(value=np.nan))

In [19]:
df_j.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1257 entries, 0 to 1256
Data columns (total 37 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   title                 1257 non-null   object        
 1   title_page            1257 non-null   object        
 2   title_rss             1257 non-null   object        
 3   description           1257 non-null   object        
 4   text_main             932 non-null    object        
 5   language              1257 non-null   category      
 6   source_domain         1257 non-null   category      
 7   filename              1257 non-null   object        
 8   url                   1257 non-null   object        
 9   image_url             1257 non-null   object        
 10  category_0            1257 non-null   object        
 11  category_1            914 non-null    object        
 12  category_2            412 non-null    object        
 13  category_3        

In [20]:
df_j.to_csv(path_to_data / "df_entities.csv", index=False)

In [141]:
# list of entities - summed up

body_ent = [x for x in df_j[[x for x in df_ent.columns if "entities" in x]]["title_entities"]]
body_ent_list = []
for x in body_ent:
    tmp = re.sub("'labels': \[(\w{3,4}) \((\d+\.?\d*)\)]", "'\g<1>': \g<2>", x)
    tmp = eval(tmp)
    tmp_list = [y for x in tmp for y in x["entities"]]
    body_ent_list.append(tmp_list)
body_org_list = {}
for x in body_ent_list:
    for y in x:
        if "ORG" in y:
            if y["text"] == "SWR":
                print(5)
            if y["text"] in body_org_list:
                body_org_list[y["text"].strip()] += 1
            else:
                body_org_list[y["text"].strip()] = 1
body_org_dict = dict(sorted(body_org_list.items(), key=lambda item: item[1], reverse=True))

5
